In [2]:
import pandas as pd
import geopandas as gpd 
import numpy as np
import xarray as xr
import os

In [22]:
gdf = gpd.read_file('../data/geodata/ucrb_selected_basins.geojson').set_index('STAID')
ds = xr.open_dataset('../data/streamflowdata/selected_hcdn_streamflow_annual.nc')

dtype('O')

In [37]:
mean_df = ds.mean(dim='water_year').to_dataframe()
# convert AREA into square kilometers
mean_df['AREA'] = mean_df['AREA'] / 1000000

# round all vales to the nearest 1
mean_df = mean_df.round(0)
# convert all numeric columns to int
for col in mean_df.columns:
    if mean_df[col].dtype != 'O':
        mean_df[col] = mean_df[col].astype(int)
# add STANAME from gdf to mean_df
mean_df['STANAME'] = gdf.STANAME
# in elevation category capitolize the first letter
mean_df['elevation_category'] = mean_df['elevation_category'].str.capitalize()
# replace medium with middle
mean_df['elevation_category'] = mean_df['elevation_category'].replace('Medium', 'Mid')
mean_df['elevation_category'] = pd.Categorical(mean_df['elevation_category'], ["Low", "Mid", "High"])

In [42]:
ordered_df = mean_df[['STANAME','AREA','mean_elevation','elevation_category','wy_q_mm','wy_ppt']].sort_values('elevation_category', )
# save to excel
ordered_df.to_csv('../data/ucrb_selected_basins_summary.csv', index=True)